In [ ]:
import torch

## 张量的赋值与索引

In [ ]:
b = torch.ones(3)   # 创建大小为3的一维张量, 用1.0填充
print(b)

print(b[1])
print(float(b[1]))  # 直接索引元素得到的还是张量, 需要转化成浮点型

In [ ]:
b[2] = 2.0
print(b)
b[2] = torch.tensor(3.)
print(b)
b[2] = torch.tensor([4.])     # 把 python list 转化成张量
# ERROR: b[2] = torch.tensor([4., 5.])
print(b)

# 三种赋值方式等价

复习python列表索引

In [ ]:
some_list = list(range(6))
print(some_list[:])
print(some_list[1:4])
print(some_list[:4])
print(some_list[:-1])
print(some_list[1:5:2])   # 1到5, 左闭右开, 步长为2

## 张量的形状

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 2.0], [6.0, 3.0]])
print('Get shape: ', points.shape)   # 3行, 2列

print(points[0,1])   # 双索引, 0号列表里的1号元素, 从外往里数方括号

## 张量维度的变换

张量的每一对方括号 **<u>向内</u>** 是一个维度

```
升维
torch.unsqueeze(
	input, 
	dim, 	     # dim 从0算起, 将要扩增的维度【必填】
	out = None)
如果dim为负, 则将会被转化dim+input.dim()+1, 即在最后增一维

降维
torch.squeeze(
	input, 
	dim, 	     # dim 从0算起, 将要挤压的维度【必填】
	out = None)
如果dim指定的维度的值为1, 则将该维度删除, 若指定的维度值不为1, 则返回原来的tensor

unsqueeze_ 和 unsqueeze 实现一样的功能, 
区别在于 unsqueeze 不会对使用 unsqueeze 的 tensor 进行改变, 想要获取 unsqueeze 后的值必须赋予个新值, 
unsqueeze_ 则会对自己改变. 
```

In [ ]:
img_t = torch.randn(3, 5, 5)        # Shape (channels, rows, columns)
print(img_t.shape, img_t)
batch_t = torch.randn(2, 3, 5, 5)   # Shape (batch, channels, rows, columns <- 这个叫作第四个维度的数量)
# print('batch_t: ',batch_t)

img_gray_naive = img_t.mean(-3)
# 消除倒数第三个维度. 假设这个维度的数量是3, 把这个维度上的每3个值叠起来用一个平均值代替. 
batch_gray_naive = batch_t.mean(-3)
print(img_gray_naive.shape)
# print(batch_gray_naive.shape, batch_gray_naive)

## 张量的求和和点乘

```
张量点乘, 对于某个维度:
若数量都不是1, 则必须数量相等才能相乘, 计算方法是: i位对i位相乘
若数量有一个是1, 则可以相乘, 计算方法是: 多数量的每一个值分别去乘单数量的值, 类似乘法分配律
```

In [ ]:
weights = torch.tensor([0.2126, 0.7152, 0.0722])
# 我们可以对相同形状的张量进行乘法运算, 所以需要对张量进行升维/降维

unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)  # 在最后(即最小维度)升两维
print(unsqueezed_weights.shape, unsqueezed_weights)

img_weights = (img_t * unsqueezed_weights)
print(img_weights.shape)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
# batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

In [ ]:
# tensor multiplication test

def mult(A, B):
    print(A, '<TIMES>', B, ' = ', A * B)

x = torch.tensor([3, 5, 7, 9])
# ERROR: x = torch.tensor([3, 5, 7])
y = torch.tensor([13, 17, 19, 23])
mult(x, y)

x = torch.tensor([3])
y = torch.tensor([13, 17, 19, 23])
mult(x, y)

x = torch.tensor([[3]])
x = torch.tensor([[3], [27]])
# ERROR: x = torch.tensor([[3], [27], [99]])
y = torch.tensor([[13, 17, 19, 23], [1, 5, 7, 11]])
mult(x, y)

x = torch.tensor([[3, 4, 1, 2], [5, 6, 7, 8]])
y = torch.tensor([[10, 11, 12, 13], [14, 15, 16, 17]])
mult(x, y)

## 张量维度的命名

In [ ]:
# 给维度命名
weights_unnamed = torch.tensor([[[0.2], [0.7]], [[0.3], [0.6]]])
weights_named = torch.tensor([[[0.2], [0.7]], [[0.3], [0.6]]], names = ['channels', 'rows', None])
print(weights_named.shape, weights_named)
weights_A = weights_unnamed.refine_names(..., 'cha', 'ro')
print(weights_A.shape, weights_A)
# ERROR: weights_B = weights_named.refine_names(..., 'cha', 'ro')
weights_B = weights_named.refine_names(..., 'rows', 'columns')
print(weights_B.shape, weights_B)
weights_C = weights_B[1][0]
print(weights_C.shape, weights_C)

# 如果本来没有名称, 三点表示在末尾处赋予名称
# 如果本来有名称, 三点后的第一个字符串应当是已经存在的名称, 改名从这里开始

In [ ]:
weights_aligned = weights_C.align_as(weights_B)
print(weights_aligned.shape, weights_aligned)
weights_mul = (weights_aligned * weights_B)
print(weights_mul.shape, weights_mul)
weights_sum = weights_mul.sum('rows')
print(weights_sum.shape, weights_sum)